In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=8a62eb25acff191b1c94f783937754b7a9f7c61cec1d1dee6d0f163a6ada7538
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.19 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 62.90 , C = 0.00010 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_16_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=891980, Tue Apr 14 11:42:49 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.794752858843314e+02 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
termination on tolfun=1e-11 (Tue Apr 14 11:42:49 2020)
final/bestever f-value = -1.794753e+02 -1.794753e+02
incumbent solution: [ 2.11915111  3.14232514 -2.8630927   0.03671253 -1.97815869  0.19474065
 -4.20034626 -0.45001135 ...]
std deviations: [2.40700098 2.40743544 2.40676239 2.40530284 2.4092594  2.40815457
 2.40571986 2.40623092 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=945201, Tue Apr 14 11:42:49 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.798314889743130e+02 1.0e+00 2.40e+00  2e+00  2e+00 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.475286, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.475286, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

termination on tolfun=1e-11 (Tue Apr 14 11:42:49 2020)
final/bestever f-value = -1.798315e+02 -1.798315e+02
incumbent solution: [ 2.33114641  4.56223951 -3.65164586  2.03986379 -0.66296944  0.20937994
 -3.53071485  0.151944   ...]
std deviations: [2.40484933 2.40466632 2.40536063 2.40401634 2.40522866 2.40438981
 2.40442751 2.40518903 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=987632, Tue Apr 14 11:42:49 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.794802747095254e+02 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 -1.795697257375277e+02 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 -1.795589501513274e+02 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.38428967] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.40677386] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   73   1241 -1.795697270081719e+02 1.4e+00 2.06e+00  2e+00  2e+00 0:03.1
  100   1700 -1.795693191253566e+02 1.4e+00 1.95e+00  2e+00  2e+00 0:04.3
  200   3400 -1.795696154245872e+02 1.7e+00 1.68e+00  1e+00  2e+00 0:08.7
  300   5100 -1.795697350285636e+02 1.9e+00 1.50e+00  1e+00  2e+00 0:13.1
  400   6800 -1.795697350830473e+02 2.2e+00 1.48e+00  1e+00  2e+00 0:17.4
  500   8500 -1.795697132675421e+02 2.4e+00 1.24e+00  7e-01  1e+00 0:21.6
  600  10200 -1.795697324263712e+02 2.8e+00 1.19e+00  6e-01  1e+00 0:26.0
  700  11900 -1.795697324062501e+02 3.2e+00 1.06e+00  5e-01  1e+00 0:30.3
  800  13600 -1.795697348276036e+02 3.5e+00 9.46e-01  4e-01  1e+00 0:34.6
  900  15300 -1.795697254922922e+02 4.0e+00 9.73e-01  4e-01  1e+00 0:38.9
 1000  17000 -1.795697348913548e+02 4.4e+00 8.20e-01  3e-01  9e-01 0:43.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.795697351661204e+02 4.9e+00 8.42e-01  3e-01  9e-01 0:47.6
 1200  20400 -1.795697324224792e+02 5.5e

...

   73   1241 -1.795696447570306e+02 1.4e+00 1.83e+00  2e+00  2e+00 0:03.2
  100   1700 -1.795694980919602e+02 1.5e+00 1.82e+00  2e+00  2e+00 0:04.3
  200   3400 -1.795694524227129e+02 1.8e+00 2.36e+00  2e+00  2e+00 0:08.6
  300   5100 -1.795697301810854e+02 2.0e+00 2.12e+00  2e+00  2e+00 0:13.0
  400   6800 -1.795685018823780e+02 2.3e+00 1.85e+00  1e+00  2e+00 0:17.3
  500   8500 -1.795697225019119e+02 2.6e+00 1.86e+00  1e+00  2e+00 0:21.7
  600  10200 -1.795695104061479e+02 2.9e+00 1.33e+00  7e-01  1e+00 0:26.0
  700  11900 -1.795696841815470e+02 3.4e+00 1.69e+00  8e-01  2e+00 0:30.3
  800  13600 -1.795697127905107e+02 3.7e+00 2.04e+00  9e-01  2e+00 0:34.6
  900  15300 -1.795697148908242e+02 4.3e+00 2.01e+00  8e-01  2e+00 0:38.9
 1000  17000 -1.795697060623026e+02 4.8e+00 2.01e+00  7e-01  2e+00 0:43.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.795697358251685e+02 5.2e+00 1.81e+00  6e-01  2e+00 0:47.5
 1200  20400 -1.795697262273519e+02 5.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.47528588] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.475286, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.475286, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

termination on tolfun=1e-11 (Tue Apr 14 14:00:58 2020)
final/bestever f-value = -1.798315e+02 -1.798315e+02
incumbent solution: [-4.83137042  2.47966057  4.94691976  3.78800559 -0.50433683 -4.94613373
 -1.48305922  2.08574464 ...]
std deviations: [2.40016118 2.40229335 2.40116189 2.40131057 2.40161852 2.40121859
 2.40135077 2.40212073 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=1001994, Tue Apr 14 14:00:58 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 -1.794653176978370e+02 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 -1.794708458355673e+02 1.0e+00 2.34e+00  2e+00  2e+00 0:00.1
    3     51 -1.795377599120085e+02 1.0e+00 2.29e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.39369977] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.45473295] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   74   1258 -1.795694584429129e+02 1.4e+00 2.09e+00  2e+00  2e+00 0:03.2
  100   1700 -1.795695906020714e+02 1.5e+00 1.98e+00  2e+00  2e+00 0:04.3
  200   3400 -1.795692563716420e+02 1.7e+00 2.35e+00  2e+00  2e+00 0:08.7
  300   5100 -1.795694498500104e+02 1.9e+00 1.98e+00  1e+00  2e+00 0:13.0
  400   6800 -1.795696415975290e+02 2.2e+00 1.74e+00  1e+00  2e+00 0:17.3
  500   8500 -1.795697306216213e+02 2.4e+00 1.72e+00  1e+00  2e+00 0:21.6
  600  10200 -1.795694315616106e+02 2.7e+00 1.29e+00  7e-01  1e+00 0:25.9
  700  11900 -1.795697095768703e+02 3.0e+00 1.88e+00  1e+00  2e+00 0:30.2
  800  13600 -1.795696439370795e+02 3.3e+00 1.86e+00  8e-01  2e+00 0:34.6
  900  15300 -1.795690957950111e+02 3.9e+00 1.75e+00  7e-01  2e+00 0:38.9
 1000  17000 -1.795697335431542e+02 4.4e+00 1.50e+00  6e-01  2e+00 0:43.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.795697351412662e+02 4.9e+00 1.83e+00  6e-01  2e+00 0:47.6
 1200  20400 -1.795697358508272e+02 5.4e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F16_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:100])
    SVM_Fun [i] = F16(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F16(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-193.16489059]), count=array([1])) 8.050303582712948
SVM
ModeResult(mode=array([-202.17860648]), count=array([1])) 12.792826349550504
ELN
ModeResult(mode=array([-196.97688443]), count=array([1])) 8.312672661734497
